In [1]:
!pip install -q langchain_community langchain-huggingface langchain_ollama langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.9/411.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.1 MB/s eta 0:00:00


In [2]:
import os
import io
import getpass
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.llms import HuggingFaceHub
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
!pip install yt-dlp webvtt-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.3 MB/s eta 0:00:00


 ### Definimos as opções:
 - 'writesubtitles': True -> baixar legendas
 - 'writeautomaticsub': True -> baixar legendas automáticas (caso as manuais não estejam disponíveis)
 - 'subtitleslangs': ['pt','pt-BR','en'] -> buscar legendas em Português e Inglês
 - 'skip_download': True -> não baixar o vídeo, só legendas e metadados
 - 'outtmpl': 'meu_video.%(ext)s' -> padrão de nomes para os arquivos gerados

In [16]:
import yt_dlp

ydl_opts = {
    'writesubtitles': True,
    'wirteautomaticsub': True,
    'subtitleslangs': ['pt', 'pt-BR', 'en'],
    'skip_download': False,
    'outtmpl': 'meu_video.%(ext)s'
}

url_video = 'https://www.youtube.com/watch?v=obKEn1KkMUI&t=1s'

In [17]:
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
  info = ydl.extract_info(url_video, download = True)

[youtube] Extracting URL: https://www.youtube.com/watch?v=obKEn1KkMUI&t=1s
[youtube] obKEn1KkMUI: Downloading webpage
[youtube] obKEn1KkMUI: Downloading tv player API JSON
[youtube] obKEn1KkMUI: Downloading ios player API JSON
[youtube] obKEn1KkMUI: Downloading m3u8 information
[info] obKEn1KkMUI: Downloading subtitles: pt-BR
[info] obKEn1KkMUI: Downloading 1 format(s): 401+251
[info] Writing video subtitles to: meu_video.pt-BR.vtt
[download] Destination: meu_video.pt-BR.vtt
[download] 100% of   35.86KiB in 00:00:00 at 640.08KiB/s
[download] Destination: meu_video.f401.mp4
[download] 100% of  235.10MiB in 00:00:10 at 22.66MiB/s  
[download] Destination: meu_video.f251.webm
[download] 100% of   26.08MiB in 00:00:00 at 31.77MiB/s  
[Merger] Merging formats into "meu_video.webm"
Deleting original file meu_video.f251.webm (pass -k to keep)
Deleting original file meu_video.f401.mp4 (pass -k to keep)


In [18]:
print("Título do vídeo:", info.get("title"))
print("Duração do vídeo:", info.get("duration"))
print("Descrição do vídeo:", info.get("description"))

Título do vídeo: TODA A MATEMÁTICA Explicada em 25 minutos.
Duração do vídeo: 1635
Descrição do vídeo: Acesse https://brilliant.org/CienciaMapeada para iniciar seu teste gratuito de 30 dias e ganhe 20% de desconto em uma assinatura premium anual.

Este vídeo foi patrocinado pela brilliant

Conteúdo

00:00 - Introdução 
01:43 - Aritmética
06:44 - Álgebra
09:10 - Geometria
12:33 - Trigonometria
14:39 - Estatítisca e probabilidade
18:15 - Brilliant
19:46 - Funções
21:57 - Limites
24:22 - Derivada
26:06 - Integral

A matemática é a linguagem das quantidades, formas, padrões e relações. Ela se organiza nas principais áreas: Aritmética, Álgebra, Geometria, Estatística e probabilidade, cálculo e infinitos outros assuntos que não cabem nesse resumo. 

Aritmética: É estudo de números e operações fundamentais como soma, subtração, multiplicação e divisão.

Álgebra: É a generalização das operações aritméticas usando variáveis e expressões para resolver equações e modelar situações.

Geometria: É 

### Verificando quais arquivos de legendas foram formados

In [20]:
import glob
sub_files = glob.glob("meu_video*.vtt")
print("Arquivos de legendas encontrados:", sub_files)

Arquivos de legendas encontrados: ['meu_video.pt-BR.vtt']


## Extraindo transcrição

In [21]:
import webvtt
import html
import re

# Nome do arquivo que o yt-dlp gerou (ajuste se vier "pt" ou outro sufixo)
arquivo_legenda = "meu_video.pt-BR.vtt"

# Lê o arquivo de legenda
legendas = webvtt.read(arquivo_legenda)

linhas_transcricao = []
for caption in legendas:
    # 1) Remove tags HTML (ex: <b>, <i>, etc.)
    texto_sem_tags = re.sub(r"<[^>]*>", "", caption.text)
    # 2) Converte entidades HTML (&nbsp;, &amp;, etc.) para caracteres reais
    texto_limpo = html.unescape(texto_sem_tags)
    # 3) Remover quebras de linha internas
    texto_limpo = texto_limpo.replace('\n', ' ')
    linhas_transcricao.append(texto_limpo)

# Junta tudo em uma única string
transcricao = "\n".join(linhas_transcricao)

# Exibe parte da transcrição limpinha
print("Trecho inicial da transcrição (sem HTML):")
print(transcricao)

Trecho inicial da transcrição (sem HTML):
Matemática: um assunto gigante e incrivelmente  diverso. Maravilhoso e ao mesmo tempo complexo.  
Mas calma, vamos começar do começo. (contagem  de números rodando Todos os assuntos surgindo  
na tela, depois do ápice da música, agrupa tudo  novamente e iniciamos com a origem da matemática). 
Desde as civilizações mais antigas,  já utilizávamos métodos para calcular,  
registrar e coletar impostos. A contagem  começou com o uso de dedos e pedras,  
evoluindo em sistemas mais complexos.  O zero, como número, surgiu na Índia,  
os números negativos, na China. Tudo se  espalhou através de outras culturas para  
Europa dando assim origem à aritmética. Enquanto isso, na antiga Mesopotâmia,  
os babilônios desenvolviam técnicas para  resolver equações simples e quadráticas.  
Porém foi durante o período islâmico medieval  que a álgebra tomou sua forma mais reconhecível,  
com o matemático persa cujo o nome é esse aí…  (Al-Khwarizmi). Ele escreveu a o

## Salvando em arquivo txt

In [22]:
with open("transcricao.txt", "w", encoding="utf-8") as f:
    f.write(transcricao)

## Carregando LLM

In [23]:
def model_gpt(model = "gpt-4o-mini", temperature = 0.1):
  llm = ChatOpenAI(model = model, temperature = temperature)
  return llm

In [24]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [25]:
llm = model_gpt()

## Criando prompt template

In [26]:
system_prompt = "Você é um assistente virtual prestativo e deve respoder a uma consulta com base na transcrição de um vídeo que será fornecido abaixo"
inputs = "Consulta: {consulta} \n Transcrição: {transcricao}"
user_prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(inputs)
prompt_template = ChatPromptTemplate.from_messages([("system", system_prompt), ("user", user_prompt)])

In [27]:
prompt_template

ChatPromptTemplate(input_variables=['consulta', 'transcricao'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Você é um assistente virtual prestativo e deve respoder a uma consulta com base na transcrição de um vídeo que será fornecido abaixo'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['consulta', 'transcricao'], input_types={}, partial_variables={}, template='<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nConsulta: {consulta} \n Transcrição: {transcricao}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'), additional_kwargs={})])

## Criação da chain

In [28]:
chain = prompt_template | llm | StrOutputParser()

## Geração da resposta

In [29]:
res = chain.invoke({"transcricao": transcricao, "consulta": "Resuma"})
print(res)

O vídeo aborda a evolução da matemática, desde suas origens nas civilizações antigas até os conceitos modernos. A matemática é apresentada como um campo vasto e complexo, começando com a contagem e o uso de números, como o zero e os números negativos, que surgiram em diferentes culturas. O desenvolvimento da álgebra é destacado, especialmente a obra de Al-Khwarizmi, e a importância da geometria na divisão de terras e construção de pirâmides.

O Renascimento é mencionado como um período crucial para a matemática, com contribuições significativas de matemáticos como Descartes e Newton. O vídeo também explora conceitos fundamentais como aritmética, álgebra, geometria, trigonometria, estatística e probabilidade, explicando suas aplicações práticas no cotidiano.

Além disso, são discutidos tópicos como funções, limites, derivadas e integrais, que são essenciais para entender o cálculo. O vídeo conclui incentivando o aprendizado da matemática de forma interativa e prática, promovendo a plata

In [30]:
res = chain.invoke({"transcricao": transcricao, "consulta": "Explique em apenas 1 frase o vídeo"})
print(res)

O vídeo explora a evolução da matemática desde suas origens nas civilizações antigas até conceitos avançados como cálculo, álgebra e estatística, destacando sua importância e aplicações no cotidiano.


In [31]:
res = chain.invoke({"transcricao": transcricao, "consulta": "Quem foram os principais matemáticos?"})
print(res)

Os principais matemáticos mencionados na transcrição do vídeo são:

1. **Al-Khwarizmi** - Matemático persa que desenvolveu a álgebra durante o período islâmico medieval.
2. **René Descartes** - Contribuiu significativamente para a álgebra e geometria.
3. **Isaac Newton** - Famoso por seus avanços em cálculo e física.
4. **Gottfried Wilhelm Leibniz** - Também fez importantes contribuições ao cálculo, de forma independente de Newton.

Esses matemáticos foram fundamentais para o desenvolvimento da matemática como a conhecemos hoje, especialmente nas áreas de álgebra, geometria e cálculo.
